# Sign Language Recognizer

In this codelab you will train a machine learning model to identify the different signs in  American Sign Language, such that given an image of a hand sign the model predicts what alphabet the sign corresponds to. For this multi class classification problem, we will use 2 different models (a Multi Layer Perceptron and a CNN) and compare the results. 

### Download the data

First let us download the data

In [0]:
# Download the dataset
!wget https://raw.githubusercontent.com/anisham197/WTMExtendedSummit/master/data/sign-language-mnist.zip

--2019-04-06 12:18:06--  https://raw.githubusercontent.com/anisham197/WTMExtendedSummit/master/data/sign-language-mnist.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32286533 (31M) [application/zip]
Saving to: ‘sign-language-mnist.zip’

sign-language-mnist 100%[===================>]  30.79M  33.8MB/s    in 0.9s    

2019-04-06 12:18:07 (33.8 MB/s) - ‘sign-language-mnist.zip’ saved [32286533/32286533]



In [0]:
mkdir input

In [0]:
!unzip sign-language-mnist.zip -d input/

Take a look at the different hand signs in American Sign Language

In [0]:
from IPython.display import Image
Image("./input/amer_sign2.png")

# About the data

The American Sign Language letter database of hand gestures represent a multi-class problem with 24 classes of letters (excluding J and Z which require motion). The dataset format is patterned to match closely with the classic MNIST. Each training and test case represents a label (0-25) as a one-to-one map for each alphabetic letter A-Z (and no cases for 9=J or 25=Z because of gesture motions). The training data (27,455 cases) and test data (7172 cases) are approximately half the size of the standard MNIST but otherwise similar with a header row of label, pixel1,pixel2....pixel784 which represent a single 28x28 pixel image with grayscale values between 0-255.

In [0]:
from IPython.display import Image
Image("./input/amer_sign3.png")

In [0]:
from IPython.display import Image
Image("./input/american_sign_language.PNG")

# Data Exploration

### Import Libraries

Import the libraries used in this codelab

In [0]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt

### **Load the dataset**

In [0]:
# Load the train and test datasets using pandas.
# Reference - https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.read_csv.html
train = pd.read_csv('./input/sign_mnist_train.csv')
test = pd.read_csv('./input/sign_mnist_test.csv')

In [0]:
# Display first n rows of the data.
# Reference - https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
train.head(10)

In [0]:
# View the dimensionality of the DataFrame.
# Reference - https://pandas.pydata.org/pandas-docs/stable/reference/frame.html
train.shape

Extract the train/test labels and convert train/test data to numpy arrays.



In [0]:
# Load the train and test labels as numpy arrays.
# Reference - https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.DataFrame.values.html
y = train['label'].values
y_test = test['label'].values

In [0]:
# Print the train and test labels and their dimensionality.
print('Train Labels:\n', y)
print('Shape of Labels:', y.shape)
print('Test Labels:\n', y_test)
print('Shape of Test Labels:', y_test.shape)

In [0]:
x = (train.drop('label', axis = 1, inplace = False)).values
x_test = (test.drop('label', axis = 1, inplace = False)).values

print('Train Feature matrix:\n', x)
print('Train Shape of Feature matrix:', x.shape)
print('Test Feature matrix:\n', x_test)
print('Test Shape of Feature matrix:', x_test.shape)

Let us plot the label distribution

In [0]:
import seaborn as sns

sns.countplot(x = y)
plt.show()

Convert a row in the dataset to it's corresponding image

In [0]:
plt.imshow(x[1].reshape(28,28), cmap='gray')
plt.title(str(y[1]) + ' - ' + chr(y[1] + ord('A')))
plt.show()

### Data Preprocessing

Since our target variables are categorical values we encode it as one hot vectors so as to easily compare it with the output of the neural network

In [0]:
num_classes = 26
y = keras.utils.to_categorical(y, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
y[0]

We also normalize the train and test data 

In [0]:
x = x.astype('float32')
x_test = x_test.astype('float32')
x /= 255
x_test /= 255

Split the data into train and validation sets

In [0]:
from sklearn.model_selection import train_test_split

# Split the training data into train and validation datasets (70-30 split).
# Reference - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.3, random_state = 101)

In [0]:
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

In [0]:
x_train[0]

In [0]:
y_train[0]

# Multi Layer Perceptron with Keras

### Training

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
from keras.callbacks import EarlyStopping

Set the hyperparameters to use in the neural network. Make sure to try different parameters

In [0]:
# Hyperparameters
learning_rate = 0.01
batch_size = 64
epochs = 20
L1_no_of_neurons = 512
L3_no_of_neurons = 256
L2_dropout = 0.1
L4_dropout = 0.1

Define the different model layers and compile it

In [0]:
model = Sequential()

# Add a densely connected NN layer with output dimensionality as 'L1_no_of_neurons' and a relu activation.
# Reference - https://keras.io/layers/core/#dense
model.add(Dense(L1_no_of_neurons, input_shape=(784,), activation='relu'))

# Add a dropout layer with 'L2_dropout' dropout rate.
# Reference - https://keras.io/layers/core/#dropout
model.add(Dropout(rate=L2_dropout))

# Let's add another dense layer with output dimensionality as 'L3_no_of_neurons' and relu activation.
model.add(Dense(L3_no_of_neurons,  activation='relu'))

# Dropout layer with L4_dropout' dropout rate.
model.add(Dropout(rate=L4_dropout))

# Add a final dense layer with output as 26 classes and softmax activation.
model.add(Dense(num_classes,  activation='softmax'))

In [0]:
# Compile the model with an SGD optimizer. Here, we will use the 'categorical_crossentropy' loss function.
# Accuracy is one of the metrics we are interested in.
# Reference - https://keras.io/models/model/#compile
# model.compile(optimizer=optimizers.SGD(lr=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# Print the model summary
# Reference - https://keras.io/models/about-keras-models/
model.summary()

Train the compiled neural network on the training data

In [0]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=2, 
                    callbacks=[EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)])

### Model Performance Visualization

Plot a graph of the training and validation accuracy

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title("Accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'])
plt.show()

Plot a graph of the training and validation loss

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','val'])
plt.show()

### Inference

Predict the output labels of the test set

In [0]:
# Predict the output labels for the test dataset.
# Reference - https://keras.io/models/model/#predict
y_pred = model.predict(x_test)

y_pred

Compute the accuracy of the model

In [0]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy of the model.
# Reference - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
accuracy_score(y_test, y_pred.round())

## Download the model for online serving

In [0]:
!pip install tensorflowjs==0.8.5

In [0]:
import tensorflowjs as tfjs

# Save the Keras model we created to use in our web application.
# Reference - 
tfjs.converters.save_keras_model(model, 'mlp.tfjs')

# CNN with Keras

### Data Preprocessing

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

Load the train and test dataset and sepearate into data and labels

In [0]:
# Load the train and test datasets.
# Reference - https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.read_csv.html
train = pd.read_csv('./input/sign_mnist_train.csv')
test = pd.read_csv('./input/sign_mnist_test.csv')

y = train['label'].values
y_test = test['label'].values
x = (train.drop('label', axis = 1, inplace = False)).values
x_test = (test.drop('label', axis = 1, inplace = False)).values

Convert the labels into one-hot vectors

In [0]:
num_classes = 26
y = keras.utils.to_categorical(y, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Normalize the data and reshape to input expected by CNN

In [0]:
x = x.astype('float32')
x_test = x_test.astype('float32')
x /=  255
x_test /= 255
x = x.reshape(x.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

Split the data into train and validation

In [0]:
from sklearn.model_selection import train_test_split

# Split the training data into train and validation datasets (70-30 split).
# Reference - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.3, random_state = 101)

Set the hyperparameters of the model

In [0]:
batch_size = 128
num_classes = 26
epochs = 10

Define the network architecture

In [0]:
model = Sequential()

# Add a 2D convolution layer with 64 filters, a 3 x 3 kernel and relu activation.
# Reference - https://keras.io/layers/convolutional/#conv2d
model.add(Conv2D(64, kernel_size=(3,3), activation = 'relu', input_shape=(28, 28 ,1) ))

# The convolution layer is followed by a maxpool layer with a 2 x 2 filter.
# Reference - https://keras.io/layers/pooling/#maxpooling2d
model.add(MaxPooling2D(pool_size = (2, 2)))

# Add another convolution layer with 64 filters, a 3 x 3 kernel and relu activation.
# Follow this with a maxpool layer with a 2 x 2 filter.
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Add a third convolution layer with 64 filters, a 3 x 3 kernel and relu activation.
# Follow this with a maxpool layer with a 2 x 2 filter.
model.add(Conv2D(64, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

# Flatten the input to the model.
# Reference - https://keras.io/layers/core/#flatten
model.add(Flatten())

# Add a dense layer with with output space as 128 and relu activation.
# Reference - https://keras.io/layers/core/#dense
model.add(Dense(128, activation = 'relu'))

# Add a droupout layer with 20% of the input units set to 0.
# Reference - https://keras.io/layers/core/#dropout
model.add(Dropout(0.20))

# Add a final dense layer with output as 26 classes and softmax activation.
model.add(Dense(num_classes, activation = 'softmax'))

Compile the model and view the summary

In [0]:
# Compile the model with an Adam optimizer. Here, we will use the 'categorical_crossentropy' loss function.
# Accuracy is one of the metrics we are interested in.
# Reference - https://keras.io/models/model/#compile
model.compile(loss = keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

# Print the model summary
# Reference - https://keras.io/models/about-keras-models/
model.summary()

Train the model on the training data

In [0]:
# Train the model providing training data, labels, validation data, epochs and batch size.
# Reference - https://keras.io/models/model/#fit
history = model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs=epochs, batch_size=batch_size)

Plot the accuracy over train and validation datasets

In [0]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title("Accuracy")
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train','val'])
plt.show()

Plot the loss over train and validation datasets

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Loss")
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train','val'])
plt.show()

### Inference

Predict the output labels of the test set

In [0]:
# Predict the output labels for the test dataset.
# Reference - https://keras.io/models/model/#predict
y_pred = model.predict(x_test)

y_pred

Compute the accuracy on the test set

In [0]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy of the model.
# Reference - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
accuracy_score(y_test, y_pred.round())

## Download the model for online serving

In [0]:
!pip install tensorflowjs==0.8.5

In [0]:
import tensorflowjs as tfjs

# Save the Keras model we created to use in our web application.
# Reference - 
tfjs.converters.save_keras_model(model, 'cnn.tfjs')